<a href="https://colab.research.google.com/github/pachterlab/BLCSBGLKP_2020/blob/master/notebooks/make_comparison_mtx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!date

Mon May 11 05:37:13 UTC 2020


In [2]:
!git clone https://github.com/pachterlab/BLCSBGLKP_2020.git

Cloning into 'BLCSBGLKP_2020'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (146/146), done.
remote: Total 161 (delta 43), reused 77 (delta 10), pack-reused 0
Receiving objects: 100% (161/161), 33.45 MiB | 10.98 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [3]:
!pip install anndata

     |████████████████████████████████| 102kB 2.6MB/s 


# Make matrix for comparison with TRB and LAMP

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable

import string
import anndata
from collections import defaultdict

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.special import expit as sigmoid


def nd(arr):
    return np.asarray(arr).reshape(-1)

def yex(ax):
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]
    
    # now plot both limits against eachother
    #ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

fsize=15

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

/usr/local/lib/python3.6/dist-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [5]:
adata = anndata.read_h5ad("BLCSBGLKP_2020/data/kb/adata.h5ad")
adata = adata[:,adata.var.sort_values("gene").index]

adata.obs["Twist_bool"] = np.logical_and(adata.obs.ATCC_viral.values==0, adata.obs.ATCC_RNA.values==0)
adata.obs["ATCC_viral_bool"] = np.logical_and(adata.obs.Twist.values==0, adata.obs.ATCC_RNA.values==0)
adata.obs["ATCC_RNA_bool"] = np.logical_and(adata.obs.Twist.values==0, adata.obs.ATCC_viral.values==0)

Trying to set attribute `.obs` of view, copying.


In [0]:
mtx = adata.layers["raw"]
gene = adata.var.gene.values
obs = adata.obs

p, l, g, c = ("Plate1", "HEK293", "N1", "Twist")

pmask = obs.plate.values==p
cmask = obs[c+"_bool"].values
lmask = obs.lysate.values==l
gzero = obs[c].values>0

m = np.logical_and.reduce([pmask, cmask, lmask])#, gzero])
vm = np.logical_or.reduce([gene==g, gene==g+"_spikein", gene=="RPP30"])

s_idx = np.where(gene==g+"_spikein")
v_idx = np.where(gene==g)

In [7]:
gene[vm]

array(['N1', 'N1_spikein', 'RPP30'], dtype=object)

In [8]:
mtx[m][:,vm].astype(int)[0:10]

array([[ 36528,  11486,      3],
       [269101,  34082,     69],
       [ 71000,   4920,     36],
       [275256,  13136,     52],
       [   328,  71991,      1],
       [   251,  42350,      5],
       [  2285, 113329,      3],
       [    59, 168450,      9],
       [  7891,  69550,      8],
       [ 27664, 129199,    150]])

# Save

In [0]:
!mkdir temporary

In [0]:
np.savetxt("temporary/mtx.csv", mtx.astype(int)[m][:,vm], delimiter=",", fmt="%.0f") 
obs["bcs"][m].to_csv("temporary/ss_bcs.txt", index=False, header=None)